chess data import

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/113.0'
}

players = {
    "Magnus Carlsen": "https://www.chess.com/games/search?fromSearchShort=1&p1=Magnus%20Carlsen&playerId=822231&page=",
    "Hikaru Nakamura": "https://www.chess.com/games/search?fromSearchShort=1&p1=Hikaru%20Nakamura&playerId=291573&page=",
    "Garry Kasparov": "https://www.chess.com/games/search?fromSearchShort=1&p1=Garry%20Kasparov&playerId=21779&page=",
    "Viswanathan Anand": "https://www.chess.com/games/search?fromSearchShort=1&p1=Viswanathan%20Anand&playerId=284082&page=",
    "Fabiano Caruana": "https://www.chess.com/games/search?fromSearchShort=1&p1=Fabiano%20Caruana&playerId=42382&page=",
    "Alexander Morozevich": "https://www.chess.com/games/search?fromSearchShort=1&p1=Alexander%20Morozevich&playerId=32913&page=",
    "Alan Pichot": "https://www.chess.com/games/search?fromSearchShort=1&p1=Alan%20Pichot&playerId=813915&page=2"
}

pgn_file = "../data/all_games.pgn"  # Path to save the combined PGN file

for player, player_url in players.items():
    game_ids = []

    for page in tqdm(range(40)):   # this attempts to grab the first 25 pages
        URL = f'{player_url}{page}'
        response = requests.get(URL, headers=headers)
        if response.status_code != 200:
            print(f'Error retrieving games for {player} on page {page}')
            break
        soup = BeautifulSoup(response.text)
        page_game_ids = [x.get('href') for x in soup.find_all('a', {'class': 'master-games-clickable-link master-games-td-user'})]
        page_game_ids = [x.split('/')[-1] for x in page_game_ids]
        game_ids.extend(page_game_ids)
        time.sleep(15)

    # Join game ids
    game_ids = ','.join(game_ids)

    DOWNLOAD_URL = f'https://www.chess.com/games/downloadPgn?game_ids={game_ids}'
    download_response = requests.get(DOWNLOAD_URL)

    with open(pgn_file, 'ab') as file:
        file.write(download_response.content)

best move cache creation

In [ ]:
import pickle

# Convert board states to FEN strings
fen_list = [board.fen() for game in board_states for board in game]

# Create an empty cache
cache = {}

# Process fen_list and populate the cache with best moves
for fen in fen_list:
    # Set the position in Stockfish
    stockfish.set_fen_position(fen)

    # Get the best move from Stockfish
    best_move = stockfish.get_best_move()

    # Add the best move to the cache
    cache[fen] = best_move

# Save the cache as a pickle file
with open("cache.pkl", "wb") as file:
    pickle.dump(cache, file)